# Reinforcement learning from (human) feedback

В этом семинаре я постараюсь поверхностно рассказать про обучение с подкреплением (Reinforcement learning) с обратной связью от человека (human feedback). Поверхносто это будет из-за того, что это новый метод, который к тому же не до конца публичный. RLHF был разработан OpenAI и именно он позволил достичь сильного прироста в качестве генеративных моделей, которое можно наблюдать в ChatGPT. OpenAI не раскрывают всех деталей того, как они этого добились. Есть только обобщенные описание того, как это все работает и большая часть научного/инженерного сообщества в NLP сейчас пытается возпроизвести RLHF. 

В семинаре мы попробуем использовать библиотеку TRL - https://github.com/lvwerra/trl 
Это библиотека связана с Huggingface и поэтому ей удобно пользоваться. (Еще есть похожая библиотека - https://github.com/CarperAI/trlx, я не до конца понимаю насколько она связана с trl, кажется что обе развиваются) 

In [25]:
# %pip install transformers trl wandb

### Reinforcement learning 
Прежде чем разбирать RLHF давайте коротко разберем, что такое Reinforcement learning в целом. RL - это отдельная парадигма в машинном обучении наряду с supervised и unsupervised learning. RL похоже на supervised learning, но модель в нем обучается не на конкретных примерах, а используя положительную/отрицательную обратную связь на свои действия от среды. 
Например, RL применяется в играх (от шахмат до доты) и в робототехнике. В принципе можно обучить supervised модель на действиях людей в игре или в управлении роботом, но тогда получается, что модель будет учиться имитировать людей, а не искать оптимальное решение. В этих задачах можно обойтись и без людей, так как можно автоматически оценить успешность/не успешность - в игре это очки или победа, в роботике это может быть, например, перемещение предмета из точки а в точку б. Обобщоно RL можно представить вот так:

![](https://www.guru99.com/images/1/082319_0514_Reinforceme1.png)

Модель учиться предсказывать действия в конкретной ситуации так, чтобы в итоге прийти к успеху (в игре действия могут быть движение вниз или вверх, а конкретной ситуацией может быть текущий экран). Изначально модель совершает рандомные действия, но постепенно у нее накапливаются последовательности действий, которые ведут к успеху и она начинает их повторять (и избегать тех, которые ведут к поражению).
На практике это может быть очень сложно, так как часто в играх количество потенциально возможных последовательностей очень большое и модель просто не может случайно найти правильные последовательности,и требуется использовать какие-то трюки, специфичные для задачи (например, можно обучать модели играть друг против друга, чтобы можно было играть много игр парралельно в ускоренном режиме и не зависеть от людей). Сейчас RL это по большей части исследовательская область и есть даже мнения, что в итоге RL окажется не нужен (например, так считает Le Cun). Но в некоторых конкретных задачах, RL получилось успешно применить. В том числе, кажется, успешен он и для улучшение языковых моделей.

### Как можно применить RL к языковым моделям?
При генерации текста у нас нет среды, которая могла бы оценить качество текста. Точнее она есть (это человек), но использовать ее напрямую затруднительно, поэтому в RLHF создается отедльная модель, которая будет имитировать человека и оценивать сгенерированные тексты. Такая модель обучается на датасете, который размечается людьми. Сначала с помощью языковой модели генерируется какое-то количество текстов и людей просят выбрать лучшие из них, чтобы сопоставить каждому тексту какое-то число (усредненная оценка). Затем обучается модель, которая предсказывает такие оценки. И на финальном шаге с RL обучаемая генеративная модель генерирует текст и он оценивается с помощью модели обученной на обратной связи от людей. Модель дообучается так, чтобы генерированные тексты получали высокие оценки.

![](https://pbs.twimg.com/media/FfeTNJCUcAEVuo5.jpg:large)

В RLHF в реализации от OpenAI есть еще 1 важный этап - это PPO. Это метод, который используется внутри RL для дообучения модели. Важно про него понимать то, что модель не просто учится максимизировать оценку генерированных текстов, она при этом еще старается минимизировать изменения. В простом fine-tuning частая проблема это то, что дообучаемая модель со временем начинает забывать и теряет в качестве, PPO позволяет этого избежать.

### TRL

Обратная связь от людей в таком фреймворке не обязательна, главное как-то получить модель, которая будет оценивать генерируемые тексты. Можно попытаться использовать доступные модели или датасеты. Давайте попробуем использовать библиотеку trl и дообучим модель OPT-125m генерировать менее негативные тексты использую предобученную модель оценки тональности.

Схематично обучение в TRL устроено вот так:

1) Rollout - это просто генерация текстов, используя исходную языковую модель. Чтобы тексты были релеватными можно взять определенные промты и генерировать их продолжения.

2) Evaluation - это оценивание сгенерированных текстов, в нашем случае мы будет это делать с помощью предобученного классификатора тональности 

3) Optimization - это само обучение. Обратите внимание, что на этом шаге есть active_model и reference_model. Изначально это одна и та же модель, но в процессе обучения мы изменяем active_model так, чтобы она не сильно уходила от изначальное модели (reference_model)

![](https://camo.githubusercontent.com/85d00cf9bca67e33c2d1270b51ff1ac01853b26a8d6bb226b711f859d065b4a6/68747470733a2f2f68756767696e67666163652e636f2f64617461736574732f74726c2d696e7465726e616c2d74657374696e672f6578616d706c652d696d616765732f7265736f6c76652f6d61696e2f696d616765732f74726c5f6f766572766965772e706e67)

Код взят из примеров в самой библиотеке и он на torch, но опять же тут ничего сложного (huggingface, токенизация)

In [2]:
import torch
from tqdm.notebook import tqdm
import pandas as pd
from transformers import pipeline, AutoTokenizer
from datasets import load_dataset

from trl import PPOTrainer, PPOConfig, AutoModelForCausalLMWithValueHead
from trl.core import LengthSampler

In [14]:
# тут задаются гиперпараметры
config = PPOConfig(
    model_name="facebook/opt-125m",
    learning_rate=1.41e-5,
    log_with=None,
    mini_batch_size=16
)

sent_kwargs = {"return_all_scores": True, "function_to_apply": "none", "batch_size": 16}

Для генерирования текстов тут используется IMDB review датасет. От каждого текста отрезается случайно какой-то кусочек и модель должна его продолжить.

In [16]:

def build_dataset(config, dataset_name="imdb", input_min_text_length=2, input_max_text_length=8):
    tokenizer = AutoTokenizer.from_pretrained(config.model_name)
    tokenizer.pad_token = tokenizer.eos_token
    ds = load_dataset(dataset_name, split="train")
    ds = ds.rename_columns({"text": "review"})
    
    # еще есть фильтрация по длине
    ds = ds.filter(lambda x: len(x["review"]) > 200, batched=False)
    ds = ds.filter(lambda x: len(x["review"]) < 2000, batched=False)
    
    # длина кусочка определяется случайно
    input_size = LengthSampler(input_min_text_length, input_max_text_length)

    def tokenize(sample):
        sample["input_ids"] = tokenizer.encode(sample["review"])[: input_size()]
        sample["query"] = tokenizer.decode(sample["input_ids"])
        return sample

    ds = ds.map(tokenize, batched=False)
    ds.set_format(type="torch")
    return ds

In [17]:
dataset = build_dataset(config)

def collator(data):
    return dict((key, [d[key] for d in data]) for key in data[0])

Found cached dataset imdb (/root/.cache/huggingface/datasets/imdb/plain_text/1.0.0/d613c88cf8fa3bab83b4ded3713f1f74830d1100e171db75bbddb80b3345c9c0)
Loading cached processed dataset at /root/.cache/huggingface/datasets/imdb/plain_text/1.0.0/d613c88cf8fa3bab83b4ded3713f1f74830d1100e171db75bbddb80b3345c9c0/cache-cd96d6dda9f74d63.arrow
Loading cached processed dataset at /root/.cache/huggingface/datasets/imdb/plain_text/1.0.0/d613c88cf8fa3bab83b4ded3713f1f74830d1100e171db75bbddb80b3345c9c0/cache-d1953048d0486394.arrow
Loading cached processed dataset at /root/.cache/huggingface/datasets/imdb/plain_text/1.0.0/d613c88cf8fa3bab83b4ded3713f1f74830d1100e171db75bbddb80b3345c9c0/cache-0b60bdf639a5236d.arrow


In [18]:
# active_model
model = AutoModelForCausalLMWithValueHead.from_pretrained(config.model_name)
# reference_model (обратите внимание что это одна и так же модель изначально)
ref_model = AutoModelForCausalLMWithValueHead.from_pretrained(config.model_name)
tokenizer = AutoTokenizer.from_pretrained(config.model_name)
tokenizer.pad_token = tokenizer.eos_token

In [19]:
ppo_trainer = PPOTrainer(config, model, ref_model, tokenizer, dataset=dataset, data_collator=collator)

In [20]:
device = ppo_trainer.accelerator.device
if ppo_trainer.accelerator.num_processes == 1:
    device = 0 if torch.cuda.is_available() else "cpu"  # to avoid a `pipeline` bug

Для оценки тональности используется пайплайн из huggingface и предобученная модель

In [ ]:
sentiment_pipe = pipeline("sentiment-analysis", model="lvwerra/distilbert-imdb", device=device)

Для каждого текста она выдает скор негативности и положительности. Для обучения дальше будет использоваться скор положительности

In [21]:
text = "this movie was really bad!!"
sentiment_pipe(text, **sent_kwargs)

[[{'label': 'NEGATIVE', 'score': 2.3350486755371094},
  {'label': 'POSITIVE', 'score': -2.726576566696167}]]

In [22]:
text = "this movie was really good!!"
sentiment_pipe(text, **sent_kwargs)

[[{'label': 'NEGATIVE', 'score': -2.2947897911071777},
  {'label': 'POSITIVE', 'score': 2.557039737701416}]]

Еще одно место, где можно настраивать параметры. Эти параметры мы разбирали на семинаре по GPT они отвечают за генерацию.

In [23]:
gen_kwargs = {"min_length": -1, "top_k": 0.0, "top_p": 1.0, 
              "do_sample": True, "pad_token_id": tokenizer.eos_token_id}

Само обучение. Для небольших моделей оно даже не очень долгое

In [24]:
output_min_length = 4
output_max_length = 16
output_length_sampler = LengthSampler(output_min_length, output_max_length)


generation_kwargs = {
    "min_length": -1,
    "top_k": 0.0,
    "top_p": 1.0,
    "do_sample": True,
    "pad_token_id": tokenizer.eos_token_id,
}


for epoch, batch in tqdm(enumerate(ppo_trainer.dataloader), 
                         total=dataset.num_rows//ppo_trainer.dataloader.batch_sampler.batch_size):
    query_tensors = batch["input_ids"]

    #### Get response from gpt2
    response_tensors = []
    for query in query_tensors:
        gen_len = output_length_sampler()
        generation_kwargs["max_new_tokens"] = gen_len
        response = ppo_trainer.generate(query, **generation_kwargs)
        response_tensors.append(response.squeeze()[-gen_len:])
    batch["response"] = [tokenizer.decode(r.squeeze()) for r in response_tensors]

    #### Compute sentiment score
    texts = [q + r for q, r in zip(batch["query"], batch["response"])]
    pipe_outputs = sentiment_pipe(texts, **sent_kwargs)
    rewards = [torch.tensor(output[1]["score"]) for output in pipe_outputs]

    #### Run PPO step
    stats = ppo_trainer.step(query_tensors, response_tensors, rewards)
    ppo_trainer.log_stats(stats, batch, rewards)

  0%|          | 0/80 [00:00<?, ?it/s]

You're using a GPT2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/opt/conda/lib/python3.10/site-packages/transformers/pipelines/base.py:1070: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


In [29]:
pd.set_option('display.max_rows', 3000)
pd.set_option('display.max_colwidth', 5000)

Можно посмотреть на результаты, сгенериров продолжения старой и новой модель

In [28]:
#### get a batch from the dataset
bs = 16
game_data = dict()
dataset.set_format("pandas")
df_batch = dataset[:].sample(bs)
game_data["query"] = df_batch["query"].tolist()
query_tensors = df_batch["input_ids"].tolist()

response_tensors_ref, response_tensors = [], []

#### get response from gpt2 and gpt2_ref
for i in range(bs):
    gen_len = output_length_sampler()
    output = ref_model.generate(
        torch.tensor(query_tensors[i]).unsqueeze(dim=0).to(device), max_new_tokens=gen_len, **gen_kwargs
    ).squeeze()[-gen_len:]
    response_tensors_ref.append(output)
    output = model.generate(
        torch.tensor(query_tensors[i]).unsqueeze(dim=0).to(device), max_new_tokens=gen_len, **gen_kwargs
    ).squeeze()[-gen_len:]
    response_tensors.append(output)

#### decode responses
game_data["response (before)"] = [tokenizer.decode(response_tensors_ref[i]) for i in range(bs)]
game_data["response (after)"] = [tokenizer.decode(response_tensors[i]) for i in range(bs)]

#### sentiment analysis of query/response pairs before/after
texts = [q + r for q, r in zip(game_data["query"], game_data["response (before)"])]
game_data["rewards (before)"] = [output[1]["score"] for output in sentiment_pipe(texts, **sent_kwargs)]

texts = [q + r for q, r in zip(game_data["query"], game_data["response (after)"])]
game_data["rewards (after)"] = [output[1]["score"] for output in sentiment_pipe(texts, **sent_kwargs)]




/opt/conda/lib/python3.10/site-packages/transformers/pipelines/text_classification.py:104: UserWarning: `return_all_scores` is now deprecated,  if want a similar funcionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/transformers/pipelines/base.py:1070: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


,query,response (before),response (after),rewards (before),rewards (after)
0,</s>Sadly,I’m a male and this post\nThis one is kind,", it's amazing.\nIt is, and today is definitely a",0.958652,2.549457
1,</s>This movie is a,"bore, people really need to chill.\nI'd like ...",very unique my kindle game. It has hooked me ...,-1.982591,2.667962
2,</s>Quite average even by,Avalanche standards. He looked scary in three...,my standards. I also love playing The Relevance,0.055134,2.123119
3,"</s>In Le Million,",the sexual assault survivor gains only an att...,I really enjoyed giving people an opportunity...,-1.407230,2.139970
4,</s>Over several years of looking,forward to the season for the New,at both Iraq and Europhyphy,1.397924,0.839216
5,</s>Nothing new,… just an adjustment for me. It's always nice to,but I really enjoyed this episode. It was a r...,1.756747,2.705983
6,"</s>So, what's",*repeatedly broken*?\nthe servers are,your favorite thing I saw at the festival las...,-1.640984,1.032504
7,</s>Before watching this film,I'd let the guy die for Chris Hedges and Donn...,"I was impressed with it. Also, I'm pretty su...",-0.061161,2.376959
8,</s>Though not a horror film,"based on the book, I",", Strong is a very interesting",0.429971,2.325984
9,</s>This is,life\nNot really. If you have room,the most inspiring and emotionalweethed momen...,-0.167638,2.791420


Видно, что модель теперь генерирует в основном только положительные тексты, даже если промт намекает на негативную тональность.